# Décryptage des Inégalités Éducatives: Une Analyse Temporelle de l'IPS Public vs. Privé

Bienvenue dans ce notebook Jupyter, conçu pour plonger au cœur des dynamiques sociales de notre système éducatif français. Notre objectif est d'analyser l'évolution de l'Indice de Position Sociale (IPS) des établissements scolaires, un indicateur clé reflétant le contexte socio-culturel des élèves. Cette analyse est rendue possible grâce au jeu de données `fr-en-ips_ecoles_v2` disponible sur le portail Open Data du Ministère de l'Éducation Nationale et de la Jeunesse, accessible via l'URL source: [https://data.education.gouv.fr/api/explore/v2.1/catalog/datasets/fr-en-ips_ecoles_v2/exports/csv?use_labels=true](https://data.education.gouv.fr/api/explore/v2.1/catalog/datasets/fr-en-ips_ecoles_v2/exports/csv?use_labels=true). Ce dataset détaillé, couvrant les rentrées scolaires de 2016-2017 à 2021-2022, fournit l'IPS pour chaque établissement, ainsi que des informations cruciales comme son académie, son département, et surtout, son secteur (public ou privé sous contrat).

La question centrale de notre investigation porte sur les disparités potentielles d'IPS entre les établissements scolaires publics et privés sous contrat, et leur évolution au fil du temps. En désagrégeant l'IPS moyen par secteur et par année scolaire, nous cherchons à visualiser et quantifier l'ampleur de ces inégalités sociales. L'aperçu obtenu est fondamental pour comprendre les dynamiques socio-économiques à l'œuvre dans nos réseaux d'enseignement et pourra servir de levier à la réflexion sur les politiques publiques visant à promouvoir une éducation plus équitable. Ce thème s'inscrit pleinement dans les enjeux des **Inégalités Éducation IPS**.

## Méthodologie: De la Donnée Brute à l'Analyse Croisée

Pour mener à bien cette analyse, nous avons adopté une approche structurée en plusieurs étapes, optimisée pour l'exploration de données.

Tout d'abord, les données brutes issues du dataset ont fait l'objet d'un nettoyage préliminaire. Nous avons extrait l'année scolaire de la colonne `"Rentrée scolaire"` et converti l'Indice de Position Sociale (`"IPS"`) en un format numérique (`DOUBLE`), tout en filtrant les entrées non valides pour assurer la robustesse de nos calculs.

Ensuite, l'étape clé a consisté à calculer l'IPS moyen pour chaque secteur (public et privé sous contrat) et pour chaque année scolaire disponible. Cette agrégation nous a permis d'obtenir une vision synthétique de l'évolution de ces indicateurs.

Enfin, pour mettre en lumière les disparités, nous avons croisé ces moyennes annuelles afin de calculer l'écart entre l'IPS moyen du public et celui du privé sous contrat. Les résultats de cette agrégation ont été ordonnés chronologiquement, préparant ainsi le terrain pour une visualisation claire et percutante des tendances et des écarts, permettant de déceler si et comment les inégalités socio-économiques entre ces deux grands réseaux éducatifs se sont accentuées, réduites ou maintenues au cours des années.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.education.gouv.fr/api/explore/v2.1/catalog/datasets/fr-en-ips_ecoles_v2/exports/csv?use_labels=true", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH cleaned_data AS (
    SELECT
        SUBSTRING("Rentrée scolaire", 1, 4) AS annee_scolaire,
        "Secteur" AS secteur,
        CAST("IPS" AS DOUBLE) AS ips_value
    FROM
        loaded_dataset
    WHERE
        CAST("IPS" AS DOUBLE) IS NOT NULL -- Filtre les lignes où IPS n'est pas un nombre valide
),
yearly_sector_ips AS (
    SELECT
        annee_scolaire,
        secteur,
        AVG(ips_value) AS avg_ips
    FROM
        cleaned_data
    GROUP BY ALL
)
SELECT
    t1.annee_scolaire,
    MAX(CASE WHEN t1.secteur = 'public' THEN t1.avg_ips ELSE NULL END) AS ips_moyen_public,
    MAX(CASE WHEN t1.secteur = 'privé sous contrat' THEN t1.avg_ips ELSE NULL END) AS ips_moyen_prive_sous_contrat,
    (MAX(CASE WHEN t1.secteur = 'public' THEN t1.avg_ips ELSE NULL END) - MAX(CASE WHEN t1.secteur = 'privé sous contrat' THEN t1.avg_ips ELSE NULL END)) AS ecart_ips_public_prive
FROM
    yearly_sector_ips AS t1
GROUP BY
    t1.annee_scolaire
ORDER BY
    t1.annee_scolaire """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 6 lignes


,annee_scolaire,ips_moyen_public,ips_moyen_prive_sous_contrat,ecart_ips_public_prive
0,2016,101.015046,111.243301,-10.228255
1,2017,101.256790,111.556445,-10.299655
2,2018,101.244216,111.681919,-10.437703
3,2019,101.249284,112.031635,-10.782351
4,2020,101.177744,112.246411,-11.068667


## 📈 Visualisation

La visualisation utilise Plotly Express pour créer des graphiques en ligne avec facettes (petits multiples), ce qui permet de visualiser l'évolution temporelle de plusieurs indicateurs d'IPS distincts côte à côte. Ce choix de représentation est idéal pour comparer des tendances qui peuvent avoir des échelles de valeurs très différentes, offrant une clarté visuelle sans confusion. Plotly Express est particulièrement adapté pour ce type de visualisation grâce à sa capacité à gérer le "tidy data" et à générer des graphiques interactifs avec un minimum de code.

In [4]:
import pandas as pd
import duckdb as ddb

import pandas as pd
import plotly.express as px

# On s'assure que les données sont triées par année pour un tracé chronologique correct
df = df.sort_values('annee_scolaire')

# On transforme les données du format large au format long ("tidy data").
# C'est la structure idéale pour créer des facettes avec Plotly Express.
df_long = pd.melt(df, 
                  id_vars=['annee_scolaire'], 
                  value_vars=['ips_moyen_public', 'ips_moyen_prive_sous_contrat', 'ecart_ips_public_prive'],
                  var_name='indicateur', 
                  value_name='valeur')

# On renomme les indicateurs pour des titres de facettes plus clairs
noms_indicateurs_map = {
    'ips_moyen_public': 'IPS Moyen Public',
    'ips_moyen_prive_sous_contrat': 'IPS Moyen Privé Sous Contrat',
    'ecart_ips_public_prive': 'Écart IPS Public / Privé'
}
df_long['indicateur'] = df_long['indicateur'].map(noms_indicateurs_map)

# Création du graphique en lignes avec des facettes pour chaque indicateur
# Les facettes permettent de comparer les tendances sans que les différentes échelles de valeur n'interfèrent
dataviz = px.line(
    df_long,
    x='annee_scolaire',
    y='valeur',
    facet_row='indicateur',
    color='indicateur', # Permet d'attribuer une couleur distincte à chaque indicateur
    markers=True, # Ajoute des points sur chaque observation
    title="Inégalités Éducation IPS : Évolution des Indicateurs IPS Publics et Privés Sous Contrat",
    labels={
        "annee_scolaire": "Année Scolaire",
        "valeur": "Valeur de l'indicateur",
        "indicateur": "Indicateur"
    }
)

# Personnalisation de l'apparence du graphique pour une meilleure lisibilité
dataviz.update_layout(
    showlegend=False,  # La légende est redondante car les titres des facettes sont explicites
    font=dict(family="Arial, sans-serif", size=12),
    margin=dict(l=80, r=20, t=50, b=20) # Ajuste les marges pour le titre de l'axe Y global
)

# On s'assure que chaque facette a sa propre échelle sur l'axe Y et on supprime les titres Y individuels
dataviz.update_yaxes(matches=None, title_text="")

# On nettoie les titres des facettes qui par défaut sont "indicateur=Nom de l'indicateur"
dataviz.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# On ajoute un titre global unique pour l'axe Y à gauche du graphique
dataviz.add_annotation(
    x=-0.1, 
    y=0.5,
    text="Valeur IPS",
    textangle=-90,
    xref="paper", 
    yref="paper",
    font=dict(size=14),
    showarrow=False
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250801_224535.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250801_224535.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250801_224535.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250801_224535.png


--> Image Plotly sauvegardée avec succès.
